In [4]:
import sys
import shutil
import os
import gzip
import bz2
from pathlib import Path

**Pileline**
* Take the SRR names grouped by the organism name (sort-u on column 6)
* Get the relevant FTP for reference genome
 * Also need to grab the genome accession (column 6 of the assembly summary)
* Use annogesic to create a project directory under the genera name with the genome accession as the dir name
* Align the reads to the genome using 

In [13]:
to_update = True

In [40]:
def get_lib_designation(file, tex_pos_terms, tex_neg_terms):
    
    global to_update
    
    search_name = file
    
    if "TEX" in file:
        dest_folder = "tex_notex"
        lib_type = ""
        for pos in tex_pos_terms:
            if pos in search_name:
                lib_type = "tex"
                search_name = search_name.replace(pos, "")
                break
        if lib_type == "":
            for neg in tex_neg_terms:
                if neg in search_name:
                    lib_type = "notex"
                    search_name = search_name.replace(neg, "")
                    break

        if lib_type == "":
            print(f"Could not find TEX designation for file: {file}")
            
            if not to_update:
                return 1
            
            resp = "."
            while resp.lower() not in ["", "y", "n"]:
                resp = choice = input("Want to add new terms (y/n/never)?")
            
            if resp == "n":
                return 1
            elif resp == "never":
                to_update = False
                return 1
            
            pos_new = input("Add to the positive terms: ")
            neg_new = input("Add to the negative terms: ")

            if pos_new != "":
                tex_pos_terms.append(pos_new)
            if neg_new != "":
                tex_neg_terms.append(neg_new)
            return get_lib_designation(file, tex_pos_terms, tex_neg_terms)


    else:
        lib_type = "frag"
        dest_folder = "fragment"
        
    return lib_type, dest_folder, search_name, tex_pos_terms, tex_neg_terms

In [39]:
def get_strand_designation(search_name, forward_terms, reverse_terms):
    
    global to_update
    strand = ""

    for pos in forward_terms:
        if pos in search_name:
            strand = "+"
            search_name = search_name.replace(pos, "")
            break

    if strand == "":
        for neg in reverse_terms:
            if neg in search_name:
                strand = "-"
                search_name = search_name.replace(neg, "")
                break
    if strand == "":
        print(f"Could not find strand designation for file: {file}")
        if not to_update:
            return 1

        resp = "."
        while resp.lower() not in ["", "y", "n"]:
            resp = choice = input("Want to add new terms (y/n/never)?")

        if resp == "n":
            return 1
        elif resp == "never":
            to_update = False
            return 1

        pos_new = input("Add to the positive terms: ")
        neg_new = input("Add to the negative terms: ")

        if pos_new != "":
            forward_terms.append(pos_new)
        if neg_new != "":
            reverse_terms.append(neg_new)
        return get_strand_designation(search_name, forward_terms, reverse_terms)
    
    return strand, search_name, forward_terms, reverse_terms
            

In [29]:
import re
sample_id = "GSM3162640-V583_5_-TEX_div_by_17940710.0_multi_by_1000000.0_reverse.wig"
ids = re.compile("ID-[0-9]*-")
gsm = re.compile("GSM[0-9]*[-,_]")

ids.findall(sample_id)

[]

In [42]:
wigs_dir_path = Path("/scratch/jho5ze/bionets/operons/geo_output/")
annogesic_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/operons/annogesic")
path_to_wigs = "input/wigs"
org_dirs = os.listdir(wigs_dir_path)

tex_neg_terms = ["minus_TEX", "minusTEX", "-TEX", "TEX_minus", "TEX_neg", "TEX-untreated", "with_TEX"]
tex_pos_terms = ["plus_TEX", "plusTEX", "+TEX", "TEX_plus", "TEX_pos", "TEX-treated", "without_TEX"]

forward_terms = [".+.", "forward", "plus", "FWD"]
reverse_terms = [".-.", "reverse", "minus", "REV"]

for org in ["test"]: #org_dirs:
    org_path = wigs_dir_path / org
    os.chdir(org_path)
    treatments = []
    for file in os.listdir(org_path):
        dest_file = file
        if "chip" in file.lower() or "plasmid" in file.lower() or "TAP" in file:
#             os.remove(file)
            continue
        
        if file.endswith(".gz") or file.endswith(".bz2"):
            method, dest_file = (gzip, file[:-3]) if file.endswith(".gz") else (bz2, file[:-4])
            with open(file, "rb") as src, open(dest_file, "wb") as dest:
                dest.write(method.decompress(src.read()))
        
#         search_name = file
        
#         if "TEX" in file:
#             dest_folder = annogesic_path/org/path_to_wigs/"tex_notex"
#             lib_type = ""
#             for pos in tex_pos_terms:
#                 if pos in search_name:
#                     lib_type = "tex"
#                     search_name = search_name.replace(pos, "")
#                     break
#             if lib_type == "":
#                 for neg in tex_neg_terms:
#                     if neg in search_name:
#                         lib_type = "notex"
#                         search_name = search_name.replace(neg, "")
#                         break
                        
#             if lib_type == "":
#                 print(f"Could not find TEX designation for file: {file}")
#                 continue
            
            
#         else:
#             lib_type = "frag"
#             dest_folder = annogesic_path/org/path_to_wigs/"fragment"

        lib_results = get_lib_designation(file, tex_pos_terms, tex_neg_terms)
        if lib_results == 1:
            continue
        else:
            lib_type, dest_folder, search_name, tex_pos_terms, tex_neg_terms = lib_results
        
        
        print(lib_type)
        
        search_name = search_name.lower()
        
        ids = re.compile("ID-[0-9]*-")
        gsm = re.compile("GSM[0-9]*[-,_]")
        
        for i in ids.findall(search_name):
            search_name = search_name.replace(i)
        for i in gsm.findall(search_name):
            search_name = search_name.replace(i)
        
#         strand = ""
        
#         for pos in forward_terms:
#             if pos in search_name:
#                 strand = "+"
#                 search_name = search_name.replace(pos, "")
#                 break
        
#         if strand == "":
#             for neg in reverse_terms:
#                 if neg in search_name:
#                     strand = "-"
#                     search_name = search_name.replace(neg, "")
#                     break
#         if strand == "":
#             print(f"Could not find strand designation for file: {file}")
#             continue

        strand_results = get_strand_designation(search_name, forward_terms, reverse_terms)
        if strand_results == 1:
            continue
        else:
            strand, search_name, forward_terms, reverse_terms = strand_results
            
        if search_name not in treatments:
            treatments.append(search_name)
            
        condition = str(treatments.index(search_name))
        
        print(":".join([dest_file, lib_type, condition, "a", strand]))
        
        """
        ToDo:
        Add in some stuff to get a more consistent sample representation and to ensure that there are 4 per TEX 
        and two per fragment with the proper designations
        """
        
        break
    break
        

Could not find TEX designation for file: GSM3162643_ID-005278-0054_1_plus_a_TEX_reverbose_time.wig


Want to add new terms (y/n/never)? y
Add to the positive terms:  plus_a_TEX
Add to the negative terms:  


tex
Could not find strand designation for file: GSM3162643_ID-005278-0054_1_plus_a_TEX_reverbose_time.wig


Want to add new terms (y/n/never)? 
Add to the positive terms:  reverbose
Add to the negative terms:  


GSM3162643_ID-005278-0054_1_plus_a_TEX_reverbose_time.wig:tex:0:a:+
